<a href="https://colab.research.google.com/github/paintedpotato/extractsermons/blob/main/Extract_PrMakandiwa_Sermons_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yt-dlp pandas tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.0/182.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 89.6 MB/s eta 0:00:00


In [2]:
import subprocess
from pathlib import Path
import pandas as pd
from tqdm import tqdm

# =========================
# CONFIG
# =========================
CHANNEL_URL = "https://www.youtube.com/channel/UCF9jrwYsvw9UlSvqY6VkQbg"
OUT_DIR = "transcripts"
OUTPUT_CSV = "all_transcripts.csv"
LANG = "en"

# =========================
# HELPERS
# =========================
def run(cmd):
    return subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

def get_video_ids(channel_url):
    print("Fetching video list...")
    result = subprocess.check_output(
        f'yt-dlp --flat-playlist --print id "{channel_url}"',
        shell=True
    ).decode().splitlines()
    return result

def download_captions(video_ids, out_dir):
    Path(out_dir).mkdir(exist_ok=True)

    success = 0
    failed = 0

    for vid in tqdm(video_ids, desc="Downloading captions", unit="video"):
        url = f"https://www.youtube.com/watch?v={vid}"

        cmd = (
            f'yt-dlp --skip-download '
            f'--write-auto-sub --sub-lang {LANG} '
            f'-o "{out_dir}/%(id)s.%(ext)s" "{url}"'
        )

        proc = run(cmd)

        if proc.returncode == 0:
            success += 1
        else:
            failed += 1

    print(f"\nFinished captions → success: {success}, failed: {failed}")

def parse_vtt(vtt_file, video_id):
    rows = []
    timestamp = None

    with open(vtt_file, encoding="utf-8") as f:
        for line in f:
            line = line.strip()

            if "-->" in line:
                timestamp = line.split(" --> ")[0]

            elif line and not line.startswith("WEBVTT"):
                rows.append({
                    "video_id": video_id,
                    "timestamp": timestamp,
                    "text": line
                })

    return rows

def combine_captions(folder, output_csv):
    all_rows = []

    files = list(Path(folder).glob("*.vtt"))

    for file in tqdm(files, desc="Parsing VTT files", unit="file"):
        vid = file.stem.split(".")[0]
        all_rows.extend(parse_vtt(file, vid))

    df = pd.DataFrame(all_rows)
    df.to_csv(output_csv, index=False)

    print(f"\nSaved CSV → {output_csv}")
    print(f"Total rows: {len(df)}")

# =========================
# RUN PIPELINE
# =========================
video_ids = get_video_ids(CHANNEL_URL)
print(f"Found {len(video_ids)} videos\n")

download_captions(video_ids, OUT_DIR)
combine_captions(OUT_DIR, OUTPUT_CSV)


Fetching video list...
Found 484 videos




Finished captions → success: 480, failed: 4


Parsing VTT files: 100%|██████████| 465/465 [00:03<00:00, 127.15file/s]



Saved CSV → all_transcripts.csv
Total rows: 2106734


In [3]:
import pandas as pd

INPUT_CSV = "all_transcripts.csv"
OUTPUT_CSV = "mary_mentions_ranked.csv"

# Load transcript dataset
df = pd.read_csv(INPUT_CSV)

# Ensure text column exists
df["text"] = df["text"].astype(str)

# -----------------------------
# Filter mentions of "Mary"
# -----------------------------
mary_df = df[df["text"].str.contains(r"\bmary\b", case=False, regex=True)].copy()

# Build video URL column
mary_df["video_url"] = "https://www.youtube.com/watch?v=" + mary_df["video_id"]

# -----------------------------
# Count mentions per video
# -----------------------------
counts = (
    mary_df.groupby("video_id")
    .size()
    .reset_index(name="mary_mentions")
)

# Merge counts back into dataset
mary_df = mary_df.merge(counts, on="video_id")

# Rank videos by number of mentions
mary_df = mary_df.sort_values(
    by="mary_mentions",
    ascending=False
)

# Reorder columns for readability
mary_df = mary_df[
    ["video_id", "video_url", "mary_mentions", "timestamp", "text"]
]

# Save output
mary_df.to_csv(OUTPUT_CSV, index=False)

print("Saved:", OUTPUT_CSV)
print("Total Mary mentions:", len(mary_df))
print("Videos containing Mary:", counts.shape[0])

mary_df.head(20)


Saved: mary_mentions_ranked.csv
Total Mary mentions: 430
Videos containing Mary: 60


,video_id,video_url,mary_mentions,timestamp,text
231,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:06:47.270,&gt;&gt; City of Mary.
232,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:06:47.280,&gt;&gt; City of Mary.
233,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:07:19.839,the<01:07:20.160><c> city</c><01:07:21.119><c>...
234,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:07:24.470,the city of Mary so that he would go to
235,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:07:24.480,the city of Mary so that he would go to
236,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:07:31.680,the<01:07:31.920><c> city</c><01:07:33.039><c>...
284,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:41:06.149,you. That's the one thing that Mary
283,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:41:03.280,you.<01:41:04.000><c> That's</c><01:41:04.239>...
282,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:39:22.560,Mary
281,FCLU0TzeUfs,https://www.youtube.com/watch?v=FCLU0TzeUfs,83,01:39:22.550,Mary
